<center><h1>Data Cleaning with Pandas</h1><center>

In [1]:
import pandas as pd

## Dealing with multiple files

We can combine the use of `glob`, a Python library for working with files, with pandas to organize multiple datasets better. `glob` can open multiple files by using regex matching to get the filenames:
```python
import glob

files = glob.glob("file*.csv") #all files starting with "file"

df_list = []
for filename in files:
    data = pd.read_csv(filename)
    df_list.append(data)

df = pd.concat(df_list)

print(files)
```

In [2]:
import glob

student_files = glob.glob('exams\exams[0-9].csv') #each csv has 100 rows (students)
df_list = []

for files in student_files:
    data = pd.read_csv(files)
    df_list.append(data)
    
students = pd.concat(df_list)
display(students)

,id,full_name,gender_age,fractions,probability,grade
0,0,Barrett Feragh,M14,76%,72%,9th grade
1,1,Llewellyn Keech,M14,83%,NaN,12th grade
2,2,Llewellyn Keech,M14,83%,NaN,12th grade
3,3,Terrell Geri,M15,80%,86%,11th grade
4,4,Gram Hallewell,M14,67%,78%,10th grade
...,...,...,...,...,...,...
95,95,Maxi Dew,F16,77%,71%,10th grade
96,96,Jewell Boas,F15,57%,90%,12th grade
97,97,Lebbie Twine,F17,72%,91%,12th grade
98,98,Garek Culbert,M14,64%,NaN,11th grade


## Reshaping the data

In [3]:
import sys
sys.path.insert(1, "C:\\Users\\USER\\FOLDA\\Data Cleaning\\exams")
from students import students
students.head()

,full_name,gender_age,fractions,probability,grade
0,Moses Kirckman,M14,69%,89%,11th grade
1,Timofei Strowan,M18,63%,76%,11th grade
2,Silvain Poll,M18,69%,77%,9th grade
3,Lezley Pinxton,M18,NaN,72%,11th grade
4,Bernadene Saunper,F17,72%,84%,11th grade


Remember, in order to achieve data "tidiness" we want:
* Each variable as a separate column
* Each row as a separate observation

We can use `pd.melt()` to do this transformation. `.melt()` takes in a DataFrame, and the columns to unpack:

In [4]:
#First, let's look at the columns:
print(students.columns)

Index(['full_name', 'gender_age', 'fractions', 'probability', 'grade'], dtype='object')


The parameters provided are:
* `frame`: the DataFrame you want to `melt`
* `id_vars`: the column(s) of the old DataFrame to preserve
* `value_vars`: the column(s) of the old DataFrame that you want to turn into variables
* `value_name`: what to call the column of the new DataFrame that stores the values
* `var_name`: what to call the column of the new DataFrame that stores the variables

In [5]:
students = pd.melt(frame=students, id_vars=['full_name', 'gender_age', 'grade'],
                   value_vars=['fractions', 'probability'],
                   value_name='score',
                   var_name='exam')

display(students.head())

,full_name,gender_age,grade,exam,score
0,Moses Kirckman,M14,11th grade,fractions,69%
1,Timofei Strowan,M18,11th grade,fractions,63%
2,Silvain Poll,M18,9th grade,fractions,69%
3,Lezley Pinxton,M18,11th grade,fractions,NaN
4,Bernadene Saunper,F17,11th grade,fractions,72%


In [6]:
print(students.exam.value_counts())

probability    1000
fractions      1000
Name: exam, dtype: int64


## Dealing with duplicates

In [7]:
duplicates = students.duplicated()
print(duplicates)

0       False
1       False
2       False
3       False
4       False
        ...  
1995    False
1996    False
1997    False
1998    False
1999    False
Length: 2000, dtype: bool


In [8]:
print(duplicates.value_counts())

False    1976
True       24
dtype: int64


In [9]:
#updating students DataFrame with duplicates removed
students = students.drop_duplicates() #without arguments, drop_duplicates() will only drop perfectly matching duplicate rows

In [10]:
#verifying no duplicates left:
duplicates = students.duplicated()
print(duplicates.value_counts())

False    1976
dtype: int64


We can also remove rows with duplicate values in specific columns, think:
```python
fruits = fruits.drop_duplicates(subset=['item'])
```
in case we have a two of the same fruits (`items`) but with different `price` say.

## Splitting by Index

In [11]:
students.head() #We want two separate columns for gender and age.

,full_name,gender_age,grade,exam,score
0,Moses Kirckman,M14,11th grade,fractions,69%
1,Timofei Strowan,M18,11th grade,fractions,63%
2,Silvain Poll,M18,9th grade,fractions,69%
3,Lezley Pinxton,M18,11th grade,fractions,NaN
4,Bernadene Saunper,F17,11th grade,fractions,72%


In [12]:
students['gender'] = students.gender_age.str[0]
students['age'] = students.gender_age.str[1:]
students = students.drop(columns='gender_age')

In [13]:
students.head()

,full_name,grade,exam,score,gender,age
0,Moses Kirckman,11th grade,fractions,69%,M,14
1,Timofei Strowan,11th grade,fractions,63%,M,18
2,Silvain Poll,9th grade,fractions,69%,M,18
3,Lezley Pinxton,11th grade,fractions,NaN,M,18
4,Bernadene Saunper,11th grade,fractions,72%,F,17


## Splitting by Character
Now we want to split the `full_name` column into `first_name` and `last_name`:

In [14]:
""" 
#Working alternative:
students['first_name'] = students.full_name.apply(
    lambda x: x.split(' ')[0]
    )

students['last_name'] = students.full_name.apply(
    lambda x: x.split(' ')[1]
    )
"""
name_split = students.full_name.str.split(" ") #Splitting by whitespace (could be . - _ , )
students["first_name"] = name_split.str.get(0)
students["last_name"] = name_split.str.get(1)
students.head()

,full_name,grade,exam,score,gender,age,first_name,last_name
0,Moses Kirckman,11th grade,fractions,69%,M,14,Moses,Kirckman
1,Timofei Strowan,11th grade,fractions,63%,M,18,Timofei,Strowan
2,Silvain Poll,9th grade,fractions,69%,M,18,Silvain,Poll
3,Lezley Pinxton,11th grade,fractions,NaN,M,18,Lezley,Pinxton
4,Bernadene Saunper,11th grade,fractions,72%,F,17,Bernadene,Saunper


## Looking at types

In [15]:
print(students.dtypes)

full_name     object
grade         object
exam          object
score         object
gender        object
age           object
first_name    object
last_name     object
dtype: object


In [16]:
#Currently, this creates an error:
#print(students.score.mean())

#because the score column is formed by strings. We'll get rid of the '%' symbol and change the datatype to float64.

## String Parsing

In [18]:
students.score = students['score'].replace('\%', '', regex=True)
students.score = pd.to_numeric(students.score)

students.head() #Got rid of '%' symbol

,full_name,grade,exam,score,gender,age,first_name,last_name
0,Moses Kirckman,11th grade,fractions,69.0,M,14,Moses,Kirckman
1,Timofei Strowan,11th grade,fractions,63.0,M,18,Timofei,Strowan
2,Silvain Poll,9th grade,fractions,69.0,M,18,Silvain,Poll
3,Lezley Pinxton,11th grade,fractions,NaN,M,18,Lezley,Pinxton
4,Bernadene Saunper,11th grade,fractions,72.0,F,17,Bernadene,Saunper


Now we want to split the numeric grade from the `grade` column.

In [19]:
split_students = students['grade'].str.split('(\d+)',expand=True)
split_students.head()

,0,1,2
0,,11,th grade
1,,11,th grade
2,,9,th grade
3,,11,th grade
4,,11,th grade


In [20]:
students.grade = split_students[1]
students.grade = pd.to_numeric(students.grade)

students.age = pd.to_numeric(students.age) #Let's also change the age to int64 while we're at it
students.head()

,full_name,grade,exam,score,gender,age,first_name,last_name
0,Moses Kirckman,11,fractions,69.0,M,14,Moses,Kirckman
1,Timofei Strowan,11,fractions,63.0,M,18,Timofei,Strowan
2,Silvain Poll,9,fractions,69.0,M,18,Silvain,Poll
3,Lezley Pinxton,11,fractions,NaN,M,18,Lezley,Pinxton
4,Bernadene Saunper,11,fractions,72.0,F,17,Bernadene,Saunper


In [21]:
print(students.dtypes) #Reflects the 3 changes we made to score, grade and age columns

full_name      object
grade           int64
exam           object
score         float64
gender         object
age             int64
first_name     object
last_name      object
dtype: object


In [22]:
#We can now calculate metrics like the mean/median/mode etc, on these columns:
print("Students average score", students.score.mean())
print("Median grade", students.grade.median())
print("Students average score ", students.age.mean())

Students average score 77.69657422512235
Median grade 11.0
Students average score  16.021255060728745


## Missing Values

Syntax:
```python
bill_df = bill_df.fillna(value={"bill":bill_df.bill.mean(), "num_guests":bill_df.num_guests.mean()})
``` 
where `bill` and `num_guests` are the columns with missing values.
<br/><br/><br/><br/>
We will assume that everyone who doesn’t have a score for an exam missed the test. We want to replace all `nans` with a score of `0`. Let’s do this with the `score` column.

In [23]:
students = students.fillna(value={"score":0})
students.head()

,full_name,grade,exam,score,gender,age,first_name,last_name
0,Moses Kirckman,11,fractions,69.0,M,14,Moses,Kirckman
1,Timofei Strowan,11,fractions,63.0,M,18,Timofei,Strowan
2,Silvain Poll,9,fractions,69.0,M,18,Silvain,Poll
3,Lezley Pinxton,11,fractions,0.0,M,18,Lezley,Pinxton
4,Bernadene Saunper,11,fractions,72.0,F,17,Bernadene,Saunper


In [24]:
#Comparing new average after filling in the missing scores:
print("New Students average score", students.score.mean())

New Students average score 72.30971659919028
